In [1]:
# only run once
!conda create -n resnet_fl_2host python=3 anaconda -y

Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /usr/local/anaconda3/envs/resnet_fl_2host

  added / updated specs:
    - anaconda
    - python=3


The following NEW packages will be INSTALLED:

  alabaster          pkgs/main/noarch::alabaster-0.7.12-py_0
  anaconda           pkgs/main/osx-64::anaconda-2020.11-py38_0
  anaconda-client    pkgs/main/osx-64::anaconda-client-1.7.2-py38_0
  anaconda-project   pkgs/main/noarch::anaconda-project-0.8.4-py_0
  applaunchservices  pkgs/main/noarch::applaunchservices-0.2.1-py_0
  appnope            pkgs/main/osx-64::appnope-0.1.0-py38_1001
  appscript          pkgs/main/osx-64::appscript-1.1.1-py38haf1e3a3_0
  argh               pkgs/main/osx-64::argh-0.26.2-py38_0
  argon2-cffi        pkgs/main/osx-64::argon2-cffi-20.1.0-py38haf1e3a3_1
  asn1crypto         pkgs/main/noarch::asn1crypto-1.4.0-py_0
  astroid            p

Preparing transaction: done
Verifying transaction: \ WARNING conda.core.path_actions:verify(963): Unable to create environments file. Path not writable.
  environment location: /Users/nicolai/.conda/environments.txt

done
Executing transaction: / WARNING conda.core.envs_manager:register_env(52): Unable to register environment. Path not writable or missing.
  environment location: /usr/local/anaconda3/envs/resnet_fl_2host
  registry file: /Users/nicolai/.conda/environments.txt
done
#
# To activate this environment, use
#
#     $ conda activate resnet_fl_2host
#
# To deactivate an active environment, use
#
#     $ conda deactivate



In [1]:
# activate conda environment to access pysyft
!source /usr/local/anaconda3/bin/activate resnet_fl_2host

In [3]:
# not necessary after 2nd run
!conda install -c pytorch pytorch -y

Solving environment: done

# All requested packages already installed.



In [4]:
# not necessary after 2nd run
!pip install syft==0.2.9

In [2]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/anaconda3/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.4.so'


In [4]:
hook = sy.TorchHook(torch)
host1 = sy.VirtualWorker(hook, id="host1")
host2 = sy.VirtualWorker(hook, id="host2")

In [7]:
class Arguments():
    def __init__(self):
        self.batch_size = 4
        self.test_batch_size = 100
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = True

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [45]:
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    test_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    train_data = datasets.ImageFolder(data_dir, transform=train_transforms)
    test_data = datasets.ImageFolder(data_dir, transform=test_transforms)
    
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)

    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    
    federated_train_loader = sy.FederatedDataLoader(
        train_data.federate((host1, host2)),
        batch_size=args.batch_size,
        #sampler=train_sampler,
        shuffle=True,
        **kwargs
    )

    test_loader = torch.utils.data.DataLoader(test_data,
        batch_size=args.test_batch_size,
        #sampler=test_sampler,    
        shuffle=True,
        **kwargs
    )
    
    return {'train': federated_train_loader, 'val': test_loader}, {'train': len(federated_train_loader), 'val':len(test_loader.dataset)}

In [46]:
dataloaders, dataset_sizes = load_split_train_test(data_dir, .2)
print(dataloaders['val'].dataset.classes, dataset_sizes)
class_names = dataloaders['val'].dataset.classes

['agricultural', 'airplane', 'baseballdiamond', 'beach', 'buildings', 'chaparral', 'denseresidential', 'forest', 'freeway', 'golfcourse', 'harbor', 'intersection', 'mediumresidential', 'mobilehomepark', 'overpass', 'parkinglot', 'river', 'runway', 'sparseresidential', 'storagetanks', 'tenniscourt'] {'train': 525, 'val': 2100}


In [16]:
torch.set_num_threads(4)

In [18]:
'''
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))
'''
'''
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    '''

In [49]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    errors = 0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                
                # NEW) send model to correct worker
                model.send(inputs.location)
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                # NEW) get model (with gradients)
                model.get()
                
                # statistics
                try:
                    running_loss += loss.get().item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                except:
                    errors += 1
                    pass
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Errors: ' + errors)
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [19]:
class Net(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        resnet = models.resnet34(pretrained=True)
        num_ftrs = resnet.fc.in_features
        resnet.fc = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(num_ftrs, n_classes)
        )
        self.base_model = resnet
        self.sigm = nn.Sigmoid()

    def forward(self, x):
        return self.sigm(self.base_model(x))

In [27]:
from torchvision import models
from torch.optim import lr_scheduler
import time
import os
import copy

model = Net(len(class_names))
model = model.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [50]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=5)
torch.save(model.state_dict(), "resnet_fl2.pt")

Epoch 0/4
----------


RuntimeError: Error, Please consider calling ".get" method instead of ".item" method, so you can be safely getting the item you need.

In [32]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer_ft, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "resnet_fl.pt")

Train Epoch: 1 [0/2112 (0%)]	Loss: -0.493185
Train Epoch: 1 [1920/2112 (91%)]	Loss: -0.518956

Test set: Average loss: -0.5051, Accuracy: 120/2100 (6%)

Train Epoch: 2 [0/2112 (0%)]	Loss: -0.488512
Train Epoch: 2 [1920/2112 (91%)]	Loss: -0.496494

Test set: Average loss: -0.5063, Accuracy: 101/2100 (5%)

Train Epoch: 3 [0/2112 (0%)]	Loss: -0.514812
Train Epoch: 3 [1920/2112 (91%)]	Loss: -0.513043

Test set: Average loss: -0.5055, Accuracy: 97/2100 (5%)

Train Epoch: 4 [0/2112 (0%)]	Loss: -0.508560
Train Epoch: 4 [1920/2112 (91%)]	Loss: -0.487546

Test set: Average loss: -0.5041, Accuracy: 98/2100 (5%)

Train Epoch: 5 [0/2112 (0%)]	Loss: -0.516424
Train Epoch: 5 [1920/2112 (91%)]	Loss: -0.531590

Test set: Average loss: -0.5080, Accuracy: 101/2100 (5%)

Train Epoch: 6 [0/2112 (0%)]	Loss: -0.474073
Train Epoch: 6 [1920/2112 (91%)]	Loss: -0.528637

Test set: Average loss: -0.5071, Accuracy: 101/2100 (5%)

Train Epoch: 7 [0/2112 (0%)]	Loss: -0.516183
Train Epoch: 7 [1920/2112 (91%)]	Loss: 